In [1]:
import network
import os
from copy import deepcopy
from main import ElasticData, StepResult, load_optimization_log, CalculationResult, CalculationSetup
from lammps_scripts import CompressionSimulation, TemperatureRange
from main import run_lammps
import convert
import torch


In [17]:
auxetic_data = "/home/sergey/python/simulator_data_gen/new_comp_lowT/"
example = os.path.join(auxetic_data, "100", "0", "7", "network.lmp")
dump = os.path.join(auxetic_data, "100", "0", "7", "dump.lammpstrj")
# example_opt = os.path.join(auxetic_data, "200", "0", "final_result.lmp")
# example_opt = network.Network.from_data_file(example_opt, include_dihedrals=False)
example = network.Network.from_data_file(example, include_dihedrals=False)
# example.set_angle_coeff(0.00)
# example.write_to_file("auxetic_zero_ang.lmp")



# with_dummies = add_pruned_bonds_back(example, example_opt)
# with_dummies.write_to_file("with_dummies.lmp")

In [5]:
compsim = CompressionSimulation(
    'x',
    example.box.x,
    "auxetic_zero_ang.lmp",
    0.03,
    temperature_range=TemperatureRange(1e-7, 1e-7, 6)
)
compsim.write_to_file("test1")

In [7]:
run_lammps("/home/sergey/python/simulator_data_gen/test1", "in.deformation")

In [ ]:
from convert import parse_dump

def get_mean_distance(graph,axis):
    """
    calculates the mean distance of the nodes along a given axis
    """
    mean_tensor = torch.mean(graph.x,dim=0)
    distances = []
    for node in graph.x:
        distances.append(abs(node[axis] - mean_tensor[axis]))
    return sum(distances)/len(distances)


def calculate_delta(last, first,  axis, device='mps'):
    """
    calculates the delta of either x or y between the first and last steps
    """
    # assume: first_d/last_d = first_box/last_box
    # get the last_box = (first_box/first_d) * last_d
    if axis == 1: # for y
        last_y = (first.box.y/get_mean_distance(first,1)) * get_mean_distance(last,1)
        delta = last_y - first.box.y
        return delta.to(device)
    else: # for x
        last_x = (first.box.x/get_mean_distance(first,0)) * get_mean_distance(last,0)
        delta = last_x - first.box.x
        return delta.to(device)
    
sim = parse_dump("/home/sergey/python/simulator_data_gen/test1/dump.lammpstrj", example, node_features="coord")

-calculate_delta(sim[-1], sim[0], 1, device='cuda') / calculate_delta(sim[-1], sim[0], 0, device='cuda')

In [ ]:
example = os.path.join(auxetic_data, "100", "0", "7", "network.lmp")
example = network.Network.from_data_file(example, include_dihedrals=False)

sim = parse_dump("/home/sergey/python/simulator_data_gen/new_comp_lowT/100/0/7/dump.lammpstrj", example, node_features="coord")

-calculate_delta(sim[-1], sim[0], 1, device='cuda') / calculate_delta(sim[-1], sim[0], 0, device='cuda')

In [2]:

def add_pruned_bonds_back(original_network: network.Network, pruned_network: network.Network):
    pruned_bonds = list(set(original_network.bonds) - set(pruned_network.bonds))
    for b in pruned_bonds:
        b.bond_coefficient = 0.0000

    with_dummies = deepcopy(pruned_network)
    combined_bonds = with_dummies.bonds + pruned_bonds
    with_dummies.bonds = combined_bonds
    with_dummies.header.bonds = len(combined_bonds)
    with_dummies.header.bond_types = len(combined_bonds)
    return with_dummies

def extract_from_opt(history, n_networks, add_pruned: bool) -> list[network.Network]:
    total_steps = len(history)
    step_size = total_steps // n_networks
    networks = [history[i].network for i in range(0, total_steps, step_size)]
    if add_pruned:
        networks = [add_pruned_bonds_back(history[0].network, networks[i]) for i in range(len(networks))]
    networks.append(history[-1].network)
    return networks

def extract_from_dump(
    path: str,
    include_angles: bool = True,
    node_features: str = "coord",
    skip: int = 1,
    network_filename: str = "network.lmp",
    dump_filename: str = "dump.lammpstrj"
):
    print(path)
    current_network = network.Network.from_data_file(
        os.path.join(path, network_filename),
        include_angles=include_angles,
        include_dihedrals=False)

    sim = convert.parse_dump(
        os.path.join(path, dump_filename),
        current_network,
        node_features=node_features,
        skip=skip
        )
    return sim

In [8]:
# sizes = [100, 120, 140, 160, 180]
sizes = [140, 160, 180]
data_path = "/home/sergey/work/auxetic_optimizer/auxetic_data/"
comp_dir = "/home/sergey/python/simulator_data_gen/new_comp_lowT_noang"
for network_size in sizes:
    local_dir = os.path.join(data_path, str(network_size))
    for subdir in os.listdir(local_dir):
        subdir_path = os.path.join(local_dir, subdir)
        history = load_optimization_log(os.path.join(subdir_path, "optimization_log.pkl"))
        networks = extract_from_opt(history, 6, add_pruned=True)

        for i, n in enumerate(networks):
            comp_sim = CompressionSimulation(
                box_size=n.box.x,
                network_filename="network.lmp",
                strain=0.03,
                temperature_range=TemperatureRange(1e-7, 1e-7, 10)
            )
            current_dir = os.path.join(comp_dir, str(network_size), str(subdir), str(i))
            print(current_dir)
            os.makedirs(current_dir, exist_ok=True)
            new = deepcopy(n)
            new.set_angle_coeff(0.00)
            new.write_to_file(os.path.join(current_dir, "network.lmp"))
            comp_sim.write_to_file(current_dir)
            run_lammps(current_dir, "in.deformation")


/home/sergey/python/simulator_data_gen/new_comp_lowT_noang/140/0/0
/home/sergey/python/simulator_data_gen/new_comp_lowT_noang/140/0/1
/home/sergey/python/simulator_data_gen/new_comp_lowT_noang/140/0/2
/home/sergey/python/simulator_data_gen/new_comp_lowT_noang/140/0/3
/home/sergey/python/simulator_data_gen/new_comp_lowT_noang/140/0/4
/home/sergey/python/simulator_data_gen/new_comp_lowT_noang/140/0/5
/home/sergey/python/simulator_data_gen/new_comp_lowT_noang/140/0/6
/home/sergey/python/simulator_data_gen/new_comp_lowT_noang/140/0/7
/home/sergey/python/simulator_data_gen/new_comp_lowT_noang/140/1/0
/home/sergey/python/simulator_data_gen/new_comp_lowT_noang/140/1/1
/home/sergey/python/simulator_data_gen/new_comp_lowT_noang/140/1/2
/home/sergey/python/simulator_data_gen/new_comp_lowT_noang/140/1/3
/home/sergey/python/simulator_data_gen/new_comp_lowT_noang/140/1/4
/home/sergey/python/simulator_data_gen/new_comp_lowT_noang/140/1/5
/home/sergey/python/simulator_data_gen/new_comp_lowT_noang/140

In [9]:
comp_dir = "/home/sergey/python/simulator_data_gen/new_comp_lowT_noang"

sizes = [100, 120, 140, 160, 180]
# sizes = [100, 120]
data = []
for network_size in sizes:
    local_dir = os.path.join(comp_dir, str(network_size))
    for subdir in os.listdir(local_dir):
        subdir_path = os.path.join(local_dir, subdir)
        print(subdir_path)
        for ld in os.listdir(subdir_path):
            current_dir = os.path.join(subdir_path, ld)
            print(current_dir)
            data.append(extract_from_dump(
                current_dir
            ))

os.chdir("/home/sergey/python/simulator_data_gen/")
torch.save(data, "opt_data_lowT_noang_full.pt")


/home/sergey/python/simulator_data_gen/new_comp_lowT_noang/100/0
/home/sergey/python/simulator_data_gen/new_comp_lowT_noang/100/0/3
/home/sergey/python/simulator_data_gen/new_comp_lowT_noang/100/0/3
/home/sergey/python/simulator_data_gen/new_comp_lowT_noang/100/0/5
/home/sergey/python/simulator_data_gen/new_comp_lowT_noang/100/0/5
/home/sergey/python/simulator_data_gen/new_comp_lowT_noang/100/0/6
/home/sergey/python/simulator_data_gen/new_comp_lowT_noang/100/0/6
/home/sergey/python/simulator_data_gen/new_comp_lowT_noang/100/0/0
/home/sergey/python/simulator_data_gen/new_comp_lowT_noang/100/0/0
/home/sergey/python/simulator_data_gen/new_comp_lowT_noang/100/0/1
/home/sergey/python/simulator_data_gen/new_comp_lowT_noang/100/0/1
/home/sergey/python/simulator_data_gen/new_comp_lowT_noang/100/0/4
/home/sergey/python/simulator_data_gen/new_comp_lowT_noang/100/0/4
/home/sergey/python/simulator_data_gen/new_comp_lowT_noang/100/0/7
/home/sergey/python/simulator_data_gen/new_comp_lowT_noang/100/0

In [10]:
torch.save(data, "opt_data_lowT_noang.pt")

In [11]:
!pwd

/home/sergey/python/simulator_data_gen
